In [36]:
from IPython.display import Image, display
import ipywidgets

In [37]:
import ee
ee.Initialize()

In [528]:
training = ee.FeatureCollection('ft:1v0w7ShdYBMVo0jJw7rRCf2-DRor6WAjsYu1v55hH')
#Remove unlabelled features
#training=training.filter(ee.Filter.greaterThanOrEquals('Riparian',0))

In [529]:
%%javascript
require.undef('geemap-widget');

define('geemap-widget', ["jupyter-js-widgets",'jquery', 'underscore'], function(widgets, $, _) {
    /**
     * A simple model to represent a layer on the map.
     *
     * @constructor
     */
    var Layer = Backbone.Model.extend({
        defaults: function() {
            return {
                config: {},
                type: undefined,
                visible: true
            };
        }
    });



    /**
     * A collection of layers.
     *
     * @constructor
     */
    var LayerCollection = Backbone.Collection.extend({
        model: Layer
    });



    /**
     * Override of the main widget model to intercept messages from Python
     * update Javascript state correctly.
     *
     * @constructor
     */
    var GoogleMapsModel = widgets.WidgetModel.extend({
        
        defaults: _.extend({}, widgets.DOMWidgetModel.prototype.defaults, {
            _model_name : 'GoogleMapsModel',
            _view_name : 'GoogleMapsView',
            _model_module : 'geemap-widget',
            _view_module : 'geemap-widget'
        }),
        
        /** @override */
        initialize: function() {
            this.listenTo(
                this, 'msg:custom', _.bind(this.handleMessage, this));
            // this.set('layers', new LayerCollection());
        },

        /**
         * Handle a message from Python.
         *
         * @param {!Object} payload Payload of the message.
         */
        handleMessage: function(payload) {
            if (!this.get('layers')) {
                this.set('layers', new LayerCollection());
            }
            switch(payload.action) {
                case 'addLayer':
                    this.get('layers').add({
                        config: payload.config,
                        type: payload.type
                    });
                    break;
                case 'removeLayer':
                    console.error('removeLayer not implemented');
                    break;
            }
        }
    });

    /**
     * A Google Maps API widget.
     *
     * @constructor
     */
    var GoogleMapsView = widgets.DOMWidgetView.extend({
        /**
         * Load the Maps API JS if needed, also prepare a deferred in case any
         * map methods are called before the map is ready.
         */
        initialize: function() {
            // Deferred to track for when the map is ready.
            this.mapsReadyDeferred = $.Deferred();

            // Dynamically adding Google Maps API JS here. Using a deferred to
            // track its load status as require returns as soon as the first
            // script loads and the Maps API triggers more scripts to append
            // which leaves a race condition where require thinks Maps API JS
            // is ready when it is not yet.
            var mapsDeferred = this.mapsDeferred = $.Deferred();
            // Another instance of this view may have already loaded the Maps
            // API JS, do not try to load it twice.
            if (window.google && window.google.maps) {
                mapsDeferred.resolve();
            } else {
                window.googleMapsCallback = function() {
                    mapsDeferred.resolve();
                };
                require(
                    ['http://maps.googleapis.com/maps/api/js?callback=googleMapsCallback'],
                    function() {},
                    function() {});
            }
        },

        /**
         * Render the map.
         */
        render: function() {
            // We must wait until the Maps API JS is ready.
            $.when(this.mapsDeferred.promise()).then(_.bind(function() {
                // Empty the views DOM. There seem to be some weird side
                // effects when you render more than one instance of this view
                // in the notebook. Cleaning the view DOM and deferring the map
                // initialization seems to work around this. It seesm almost if
                // map instances are sharing some DOM somehow.
                this.$el.empty();
                _.defer(_.bind(function() {
                    this.$map = $([
                        '<div style="height: ',
                        this.model.get('height'),
                        '; width: ',
                        this.model.get('width'),
                        ';"></div>'
                    ].join(''));
                    this.$el.append(this.$map);
                    this.map = new google.maps.Map(this.$map.get(0), {
                        center: {
                            lat: this.model.get('lat'),
                            lng: this.model.get('lng')
                        },
                        zoom: this.model.get('zoom')
                    });
                    
                    this.initializeLayersControl();

                    // Notify the map when the map container changes size via
                    // the exposed properties in the model.
                    this.listenTo(
                        this.model, 'change:height', _.bind(function() {
                            this.$map.height(this.model.get('height'));
                            google.maps.event.trigger(this.map, 'resize');
                        }, this));
                    this.listenTo(
                        this.model, 'change:width', _.bind(function() {
                            this.$map.width(this.model.get('width'));
                            google.maps.event.trigger(this.map, 'resize');
                        }, this));

                    // Bind a change in the position of the map to the model.
                    google.maps.event.addListener(
                        this.map,
                        'bounds_changed',
                        _.bind(this.syncFromMap, this));

                    // Bind a change in the model (coming from the Python side)
                    // to the location of the map.
                    this.listenTo(
                        this.model,
                        'change:lat',
                        _.bind(this.syncFromModel, this));
                    this.listenTo(
                        this.model,
                        'change:lng',
                        _.bind(this.syncFromModel, this));
                    this.listenTo(
                        this.model,
                        'change:zoom',
                        _.bind(this.syncFromModel, this));

                    // Render the initial set of layers.
                    if (!this.model.get('layers')) {
                        this.model.set('layers', new LayerCollection());
                    }
                    this.model.get('layers').each(this.buildLayer, this);

                    // Bind to changes in the layers of the model to stay in
                    // sync.
                    this.listenTo(
                        this.model.get('layers'),
                        'add',
                        _.bind(this.buildLayer, this));
                    this.listenTo(
                        this.model.get('layers'),
                        'remove',
                        function() {
                            console.error('removeLayer not implemented');
                        });

                    // Even though a google.maps.Map instance should be ready
                    // immediately, it is not. This delay lets the stack clear
                    // and initial map bounds to be set.
                    _.delay(_.bind(function() {
                        this.mapsReadyDeferred.resolve();
                    }, this), 500);
                }, this));
            }, this));
        },

        //Based Tyler's earlier implementation
        LayersControl: function(widget, controlDiv, map) {
            // Set CSS styles for the DIV containing the control
            // Setting padding to 5 px will offset the control
            // from the edge of the map.
            controlDiv.style.padding = '5px';

            // Set CSS for the control border.
            this.$controlUI = $('<div />')
                .css('backgroundColor', 'white')
                .css('borderStyle', 'solid')
                .css('borderWidth', '1px')
                .css('cursor', 'pointer')
                .css('textAlign', 'center')
                .css('visibility', 'hidden')
                .appendTo($(controlDiv));
            
            // Set CSS for the control interior.
            var $controlContents = $('<div />')
                .css('fontFamily', 'Arial,sans-serif')
                .css('fontSize', '12px')
                .css('paddingLeft', '4px')
                .css('paddingRight', '4px')
                .css('paddingTop', '0px')
                .css('paddingBottom', '0px')
                .appendTo(this.$controlUI);
            
            this.$controlTable = $('<table />')
                .append($('<tr><td colspan=2>Layers</td></tr>'))
                .appendTo($controlContents);
        },
        
        initializeLayersControl: function() {
            // Create the DIV to hold the control and call the LayersControl() constructor
            // passing in this DIV.
    
            var layersControlDiv = document.createElement('div');
            
            this.layersControl = new this.LayersControl(this, layersControlDiv, this.map);
            
            layersControlDiv.index = 1;
            this.map.controls[google.maps.ControlPosition.TOP_RIGHT].push(layersControlDiv);
        },
        
        /**
         * Sync the values from the map into the model.
         */
        syncFromMap: function() {
            this.model.set({
                lat: this.map.getCenter().lat(),
                lng: this.map.getCenter().lng(),
                zoom: this.map.getZoom()
            });
            // This is needed for the model to update the equivalent properties
            // on the Python instance of this view.
            this.model.save_changes();
        },

        /**
         * Move the map to match the values from the model.
         */
        syncFromModel: function() {
            this.map.setCenter(new google.maps.LatLng(
                this.model.get('lat'),
                this.model.get('lng')
            ));
            this.map.setZoom(this.model.get('zoom'));
        },

        /**
         * Add a layer to the map based on its model.
         *
         * @param {!Layer} layer The layer to add.
         */
        buildLayer: function(layer) {
            switch(layer.get('type')) {
                case 'geojsondata':
                    this.addGeoJsonLayer(layer.get('config').data);
                    break;
                case 'geojsonurl':
                    this.loadGeoJsonLayer(layer.get('config').url);
                    break;
                case 'kmlurl':
                    this.loadKmlLayer(layer.get('config').url);
                    break;
                case 'earthengine':
                    this.addEarthEngineLayer(
                        layer.get('config').mapid, layer.get('config').token, layer.get('config').name, layer.get('config').visible);
                    break;
            }
        },

        /**
         * Adds GeoJSON to the map.
         *
         * @param {!Object} data A GeoJSON object.
         */
        addGeoJsonLayer: function(data) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                this.map.data.addGeoJson(data);
            }, this));
        },

        /**
         * Adds a URL location of GeoJSON to the map.
         *
         * @param {string} url The URL of the GeoJSON file to load.
         */
        loadGeoJsonLayer: function(url) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                this.map.data.loadGeoJson(url);
            }, this));
        },

        /**
         * Add a KML layer to the map.
         *
         * @param {string} url The URL of the KML file to load.
         */
        loadKmlLayer: function(url) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                new google.maps.KmlLayer({
                    url: url,
                    map: this.map
                });
            }, this));
        },

        /**
         * Add an Earth Engine layer to the map.
         *
         * @param {string} mapid The id of the Earth Engine layer.
         * @param {string} token The OAuth token to authenticate with.
         */
        addEarthEngineLayer: function(mapid, token, name, visible) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                var eeMapOptions = {
                    getTileUrl: function(tile, zoom) {
                        var url = [
                            'https://earthengine.googleapis.com/map',
                            mapid,
                            zoom,
                            tile.x,
                            tile.y
                        ].join('/');
                        url += '?token=' + token;
                        return url;
                    },
                    tileSize: new window.google.maps.Size(256, 256),
                    opacity: 1.0,
                };

                // Create the overlay map type.
                var mapType = new window.google.maps.ImageMapType(eeMapOptions);

                // Overlay the Earth Engine generated layer.
                this.map.overlayMapTypes.push(mapType);
                
                
                // Update layer visibility control
                var maxSlider = 100;

                function updateOpacity() {
                    mapType.setOpacity($checkbox[0].checked ? $slider[0].value / 100.0 : 0);
                }

                var $checkbox = $('<input type="checkbox">')
                    .prop('checked', visible)
                    .change(updateOpacity);

                var $slider = $('<input type="range" />')
                    .prop('min', 0)
                    .prop('max', maxSlider)
                    .prop('value', maxSlider)
                    .css('width', '60px')
                    .on('input', updateOpacity);

                // If user doesn't specify a layer name, create a default
                if (name === null) {
                    name = 'Layer ' + this.map.overlayMapTypes.length;
                }

                var $row = $('<tr />');
                $('<td align="left" />').append($checkbox).append(name).appendTo($row);
                $('<td />').append($slider).appendTo($row);
                
                this.layersControl.$controlUI.css('visibility', 'visible');
                this.layersControl.$controlTable.append($row);
            }, this));
        }
    });

    return {
        GoogleMapsModel: GoogleMapsModel,
        GoogleMapsView: GoogleMapsView
    }
});

<IPython.core.display.Javascript object>

In [530]:
from ipywidgets import widgets
import traitlets

class GoogleMapsView(widgets.DOMWidget):
    """Google Maps API widget."""
    _model_name = traitlets.Unicode('GoogleMapsModel').tag(sync=True)
    _view_name = traitlets.Unicode('GoogleMapsView').tag(sync=True)
    _view_module = traitlets.Unicode('geemap-widget').tag(sync=True)
    _model_module = traitlets.Unicode('geemap-widget').tag(sync=True)
    lat = traitlets.CFloat(0).tag(sync=True)
    lng = traitlets.CFloat(0).tag(sync=True)
    zoom = traitlets.CInt(2).tag(sync=True)
    height = traitlets.CUnicode('300px').tag(sync=True)
    width = traitlets.CUnicode('400px').tag(sync=True)
    layers = traitlets.List([]).tag(sync=False)

    def addGeoJsonLayer(self, data):
        """Adds a dictionary of GeoJSON to the map.

        NOTE: It is likely if you are using a third party GeoJSON library you
        will have to first serialize the data into a simple dictionary before
        passing the data to this method.

        Args:
            data: A simple python dictionary of GeoJSON data.
        """
        self.send({
            'action': 'addLayer',
            'type': 'geojsondata',
            'config': {'data': data}
        })

    def loadGeoJsonLayer(self, url):
        """Adds a URL location of GeoJSON to the map.

        Args:
            url: The URL of the GeoJSON file.
        """
        self.send({
            'action': 'addLayer',
            'type': 'geojsonurl',
            'config': {'url': url}
        })

    def loadKmlLayer(self, url):
        """Adds a KML layer to the map.

        Args:
            url: The URL of the KML file.
        """
        self.send({
            'action': 'addLayer',
            'type': 'kmlurl',
            'config': {'url': url}
        })

    def addEarthEngineLayer(self, image, vis_params, name=None, visible=True):
        """Adds an Earth Engine layer to the map.

        Args:
            image: The ee.Image to display.
            vis_params: Dictionary of visualization parameters.
        """
        mapid = image.getMapId(vis_params)
        self.send({
            'action': 'addLayer',
            'type': 'earthengine',
            'config': {
                'mapid': mapid['mapid'],
                'token': mapid['token'],
                'name': name,
                'visible': visible
            }
        })

In [531]:
map1 = GoogleMapsView(lng=-119.2, lat=36.3, zoom=4, height='240px', width='800px')
#display(map1)

In [532]:
training_points = ee.Image(training.draw(color ='006600', strokeWidth= 3))

In [533]:
map1.addEarthEngineLayer(image=training_points, vis_params={})
display(map1)

In [534]:
RFseed = 1
RFsubsample = (training.randomColumn("random", RFseed).filter(ee.Filter.lt("random", 0.8)))

RFvalidation = (training.randomColumn("random", RFseed).filter(ee.Filter.gte("random", 0.8)))

In [535]:
counties = ee.FeatureCollection('ft:1sI2nrsM0zy1nbwTIiUCAG90eSr_a3jUsbbb-K5pI', 'geometry');
UT = counties.filter(ee.Filter.inList('STATE', [490]))
COWY = counties.filter(ee.Filter.inList('STATE', [80, 560]))
GRB = ee.FeatureCollection('ft:1_REjFaRegZHbKMA4QBvKzfMBHtTS8MN0xt0mnVVl')
VBo = ee.Image('users/dylanhatlas/TW_vw10m')
VB = VBo.mask(VBo.neq(1))
map1.addEarthEngineLayer(ee.Image(GRB.draw(color='00ff00',strokeWidth=1)),vis_params={},name='Green River Basin')
map1.addEarthEngineLayer(image=VB,vis_params={'color':'ffffff'},name='VB')
UT_counties = ee.Image(UT.draw(color= 'ff0000',strokeWidth=1))
map1.addEarthEngineLayer(image=UT_counties,vis_params={})
display(map1)

In [536]:
elev = ee.Image('USGS/NED');
tpi540 = elev.subtract(elev.reduceNeighborhood(ee.Reducer.mean(), ee.Kernel.circle(540, "meters")));
#map1.addEarthEngineLayer(tpi540, vis_params={'min':-200, 'max': 200,'palette':'0000ff,00ff00'}, name='tpi540', visible=True);

In [552]:
#bring in NAIP imagery for GRB extent
naip_collection = (ee.ImageCollection("USDA/NAIP/DOQQ")
            .filterBounds(GRB)
            .filterDate('2015-06-01', '2016-09-30')
            .filter(ee.Filter.listContains("system:band_names", "N")))

naip_GRB = (ee.ImageCollection("USDA/NAIP/DOQQ")
            .filterBounds(GRB)
            .filterDate('2015-06-01', '2016-09-30')
            .filter(ee.Filter.listContains("system:band_names", "N"))
            .median().clip(GRB))


#naip_GRB = (ee.ImageCollection("USDA/NAIP/DOQQ")
#            .filterBounds(GRB)
#            .filterDate('2015-06-01', '2016-09-30')
#            .filter(ee.Filter.listContains("system:band_names", ['R', 'G', 'B', 'N']))
#            .median().clip(GRB))

#.filter(ee.Filter.listContains('bands', ['R', 'G', 'B', 'N']))
#.filter(ee.Filter.listContains("system:band_names", "N"))
#            .select(['R','G','B','N'])
def band_fx(img):
    return img.set({'bands': img.bandNames()})

#Apply bandName function and select images that have Near-Infrared band
#collection = naip_GRB.map(band_fx).filter(ee.Filter.listContains('bands', 'N'))

In [539]:
#naip_GRB.getInfo()

In [540]:
ndvi_palette =  ('FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400,3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301')

In [541]:
map2 = GoogleMapsView(lng=-106.82526, lat=40.43565, zoom=16, height='500px', width='800px')
map2.addEarthEngineLayer(naip_GRB,vis_params={})
#display(map2)

In [606]:
#naipNDVI = naip_GRB.normalizedDifference(['N', 'R']);

def NDVI_fx(img):
    outImage = img
    OutImage = outImage.addBands(outImage.normalizedDifference(['N', 'R']).select([0],['nd']))
    return OutImage

naipNDVI2 = naip_collection.map(NDVI_fx).select('nd').mean()

#Derive Texture, Note that in Python arguments must be keywords rather than dictionaries 
#e.g. reducer = ee.Reducer.stdDev() as opposed to reducer:ee.Reducer.stdDev
texture = naipNDVI2.reduceNeighborhood(
    reducer = ee.Reducer.stdDev(),
    kernel = ee.Kernel.circle(5))

def EVI_expression(image):
    # L(Canopy background)
    # C1,C2(Coefficients of aerosol resistance term)
    # GainFactor(Gain or scaling factor)
    gain_factor = ee.Image(2.5);
    coefficient_1 = ee.Image(6);
    coefficient_2 = ee.Image(7.5);
    l = ee.Image(1);
    nir = image.select("N");
    red = image.select("R");
    blue = image.select("B");
    evi = image.expression(
        "Gain_Factor*((NIR-RED)/(NIR+C1*RED-C2*BLUE+L))",
        {
            "Gain_Factor":gain_factor,
            "NIR":nir,
            "RED":red,
            "C1":coefficient_1,
            "C2":coefficient_2,
            "BLUE":blue,
            "L":l
        }
    )
    return evi
#EVI = ee.ImageCollection(naip_GRB).map(EVI_fx)
#EVI = naip_collection.map(EVI_fx).mean()  
EVI = naip_collection.map(EVI_expression).mean()  
EVI.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
   'data_type': {'precision': 'double', 'type': 'PixelType'},
   'id': 'constant'}],
 'type': 'Image'}

In [574]:
#naipNDVI2.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
   'data_type': {'max': 1.0,
    'min': -1.0,
    'precision': 'float',
    'type': 'PixelType'},
   'id': 'nd'}],
 'type': 'Image'}

In [598]:
map2.addEarthEngineLayer(EVI,vis_params={'min': -1, 'max': 1, 'palette': 'FFFFFF,FF0000'})
display(map2)

In [607]:
nir = naip_GRB.select('N').unitScale(0, 1024).multiply(255).byte();
neighborhood = ee.Kernel.circle(radius=5);
glcm = nir.glcmTexture(size=2);
contrast = glcm.select('N_var');

In [608]:
#excluding EVI i.e. "constant" for now
inBands = ["R", "G", "B", "N", "nd","constant","nd_stdDev", "N_var"]
predictors = naip_GRB.addBands(naipNDVI2, names=['nd']).addBands(EVI, names=['constant']).addBands(texture, names=['nd_stdDev']).addBands(contrast,names=['N_var'])
#predictors=predictors.double()
#predictors = naip.addBands(naipNDVI, names=['nd']).addBands(EVI, names=['constant']).addBands(texture, names=['nd_stdDev']).addBands(contrast,names=['N_var'])
#predictors.getInfo()

In [609]:
RF_training = predictors.sampleRegions(RFsubsample, ['class'], 1, "EPSG:4326");
RF_testing = predictors.sampleRegions(RFvalidation, ['class'], 1, "EPSG:4326");

In [610]:
RFclassifier = (ee.Classifier.randomForest(300, 3, 10, 0.5, True)
                .train(RF_training, 'class', inBands))

trainAccuracy = RFclassifier.confusionMatrix();
#print('Resubstitution error matrix: ', trainAccuracy);
#print('Training overall accuracy: ', trainAccuracy.accuracy());
#print(trainAccuracy.accuracy())

In [611]:
#predictors.select(inBands).getInfo()
RFclassified = predictors.select(inBands).classify(RFclassifier)
veg = RFclassified.mask(VBo.eq(1).mask(RFclassified.eq(1)))

In [612]:
map3 = GoogleMapsView(lng=-106.82526, lat=40.43565, zoom=12, height='500px', width='800px')

In [613]:
map3.addEarthEngineLayer(veg,vis_params={'palette':'00cc00'})
display(map3)